In [288]:
import geopandas as gpd
import pandas as pd

import numpy as np
from pathlib import Path
from datetime import datetime

In [289]:
pd.options.display.max_columns = None
pd.options.display.max_rows = None

In [290]:
df = pd.read_excel(io=Path(Path.cwd(), 'Задание 2.xlsx'))

df.shape

(53, 14)

# Проверка качества данныx. Приведение типов и форматов

In [291]:
df.head()

,Longitude,Latitude,Customer_group_4_ID,Продажи клиенту,Продажи клиента,Группа клиента по продажам клиенту,Профит клиента,Группа клиентов по профиту,средний чек,Группа клиентов по среднему чеку,количество заявок в месяц,количество заявок в месяц-1,количество заявок в месяц-2,количество заявок в месяц +1
0,"37,566022","55,744908",Клиент 1,50,70.0,"*необходимо присвоить кластеры (python, Закон ...",10,*необходимо присвоить кластеры (python),150.0,*необходимо присвоить 3 группы (high-medium-lo...,50.0,65.00,84.5000,*необходимо построить прогноз на следующий мес...
1,"37,480038","55,707113",Клиент 2,70,98.0,NaN,24.5,NaN,300.0,NaN,4.0,2.80,1.9600,NaN
2,"37,521722","55,675178",Клиент 3,50,70.0,NaN,35,NaN,500.0,NaN,3.0,300.00,30000.0000,NaN
3,"37,612972","55,737492",Клиент 4,50,70.0,NaN,35,NaN,381.0,NaN,7.0,5.25,3.9375,NaN
4,"37,447368","55,756777",Клиент 5,50,70.0,NaN,35,NaN,150.0,NaN,12.0,7.68,4.9152,NaN


In [292]:
df.columns = df.columns.str.lower()

In [293]:
df.columns.to_list()

['longitude',
 'latitude',
 'customer_group_4_id',
 'продажи клиенту',
 'продажи клиента',
 'группа клиента по продажам клиенту',
 'профит клиента',
 'группа клиентов по профиту',
 'средний чек',
 'группа клиентов по среднему чеку',
 'количество заявок в месяц',
 'количество заявок в месяц-1',
 'количество заявок в месяц-2',
 'количество заявок в месяц +1']

In [294]:
df = df.rename(columns={
    'customer_group_4_id': 'customer',
    'продажи клиенту': 'sales_to_customer',
    'продажи клиента': 'customer_sales',
    'группа клиента по продажам клиенту': 'sales_to_customer_ABC',
    'профит клиента': 'customers_profit',
    'группа клиентов по профиту': 'customers_profit_ABC',
    'средний чек': 'AOV',
    'группа клиентов по среднему чеку': 'AOV_ABC',
    'количество заявок в месяц': 'request_cnt_m',
    'количество заявок в месяц-1': 'request_cnt_m2',
    'количество заявок в месяц-2': 'request_cnt_m3',
    'количество заявок в месяц +1': 'request_cnt_m_forecast'
})

In [295]:
df['sales_to_customer_ABC'] = np.nan
df['customers_profit_ABC'] = np.nan
df['AOV_ABC'] = np.nan
df['request_cnt_m_forecast'] = np.nan

In [296]:
df = df.dropna(subset='customer')

In [297]:
df.dtypes

longitude                  object
latitude                   object
customer                   object
sales_to_customer          object
customer_sales            float64
sales_to_customer_ABC     float64
customers_profit           object
customers_profit_ABC      float64
AOV                       float64
AOV_ABC                   float64
request_cnt_m             float64
request_cnt_m2            float64
request_cnt_m3            float64
request_cnt_m_forecast    float64
dtype: object

In [298]:
df['longitude'] = df['longitude'].str.replace(',', '.')

df['latitude'] = df['latitude'].str.replace(',', '.')

In [299]:
df = df.astype({
    'latitude':'float64',
    'longitude':'float64',
})

In [300]:
df.loc[df.sales_to_customer.apply(lambda x: isinstance(x, datetime))==True, 'sales_to_customer'] = \
    df.loc[df.sales_to_customer.apply(lambda x: isinstance(x, datetime))==True]['sales_to_customer'].astype('datetime64').dt.day

In [301]:
df.sales_to_customer = df.sales_to_customer.astype(dtype='float64')

In [302]:
df.sales_to_customer = np.ceil(df.sales_to_customer)

df.sales_to_customer = df.sales_to_customer.astype('int64')

In [303]:
df.sales_to_customer.describe().reset_index().style.format(precision=0)

,index,sales_to_customer
0,count,52
1,mean,480835
2,std,3466867
3,min,1
4,25%,50
5,50%,70
6,75%,90
7,max,25000000


In [304]:
df.loc[df.sales_to_customer.quantile(q=0.99) < df.sales_to_customer, 'sales_to_customer'] = df.sales_to_customer.median()

In [305]:
df.customer_sales = np.ceil(df.customer_sales)

df.customer_sales = df.customer_sales.astype('int64')

In [306]:
df.customer_sales.describe()

count     52.000000
mean      99.384615
std       49.920389
min        2.000000
25%       70.000000
50%       98.000000
75%      126.000000
max      350.000000
Name: customer_sales, dtype: float64

In [307]:
df.loc[df.customers_profit.apply(lambda x: isinstance(x, datetime))==True, 'customers_profit'] = \
    df.loc[df.customers_profit.apply(lambda x: isinstance(x, datetime))==True]['customers_profit'].astype('datetime64').dt.day

In [308]:
df.customers_profit = np.ceil(df.customers_profit)

df.customers_profit = df.customers_profit.astype('int64')

In [309]:
df.customers_profit.describe()

count     52.000000
mean      27.250000
std       19.987128
min        1.000000
25%       14.000000
50%       18.000000
75%       35.000000
max      102.000000
Name: customers_profit, dtype: float64

In [310]:
df.AOV.describe()

count      52.000000
mean      551.719457
std       341.607136
min        50.000000
25%       222.000000
50%       517.925000
75%       770.111029
max      1500.000000
Name: AOV, dtype: float64

In [311]:
df.request_cnt_m = np.ceil(df.request_cnt_m)
df.request_cnt_m = df.request_cnt_m.astype('int64')

df.request_cnt_m.describe()

count    52.000000
mean     13.538462
std      13.367302
min       3.000000
25%       4.000000
50%       9.000000
75%      13.000000
max      50.000000
Name: request_cnt_m, dtype: float64

In [312]:
df.request_cnt_m2 = np.ceil(df.request_cnt_m2)
df.request_cnt_m2 = df.request_cnt_m2.astype('int64')

df.request_cnt_m2.describe()

count      52.000000
mean       38.884615
std       150.564513
min         1.000000
25%         3.000000
50%         7.000000
75%        17.000000
max      1060.000000
Name: request_cnt_m2, dtype: float64

In [313]:
df.request_cnt_m3 = np.ceil(df.request_cnt_m3)
df.request_cnt_m3 = df.request_cnt_m3.astype('int64')

df.request_cnt_m3.describe()

count       52.000000
mean      2328.480769
std      13074.699432
min          1.000000
25%          2.000000
50%          5.500000
75%         20.000000
max      90000.000000
Name: request_cnt_m3, dtype: float64

In [314]:
df.head(100)

,longitude,latitude,customer,sales_to_customer,customer_sales,sales_to_customer_ABC,customers_profit,customers_profit_ABC,AOV,AOV_ABC,request_cnt_m,request_cnt_m2,request_cnt_m3,request_cnt_m_forecast
0,37.566022,55.744908,Клиент 1,50,70,NaN,10,NaN,150.000000,NaN,50,65,85,NaN
1,37.480038,55.707113,Клиент 2,70,98,NaN,25,NaN,300.000000,NaN,4,3,2,NaN
2,37.521722,55.675178,Клиент 3,50,70,NaN,35,NaN,500.000000,NaN,3,300,30000,NaN
3,37.612972,55.737492,Клиент 4,50,70,NaN,35,NaN,381.000000,NaN,7,6,4,NaN
4,37.447368,55.756777,Клиент 5,50,70,NaN,35,NaN,150.000000,NaN,12,8,5,NaN
5,37.197169,55.771022,Клиент 6,50,70,NaN,12,NaN,222.000000,NaN,24,6,2,NaN
6,37.556708,55.754942,Клиент 7,70,98,NaN,14,NaN,150.000000,NaN,13,17,22,NaN
7,37.615375,55.865067,Клиент 8,70,98,NaN,25,NaN,300.000000,NaN,3,3,2,NaN
8,37.540006,55.669169,Клиент 9,70,98,NaN,17,NaN,500.000000,NaN,7,70,3,NaN
9,37.780844,55.792342,Клиент 10,70,98,NaN,14,NaN,381.000000,NaN,9,7,6,NaN


# ABC-анализ

### Продажи клиенту

In [315]:
df = df.sort_values(by=['sales_to_customer'], ascending=False)

df['perc_of_total'] = df['sales_to_customer'] / df['sales_to_customer'].sum(axis=0)

df['perc_of_total_cum'] = df['perc_of_total'].cumsum(axis=0)

df.loc[df['perc_of_total_cum'] <= 0.8, 'sales_to_customer_ABC'] = 'A'

df.loc[
    (df['perc_of_total_cum'] > 0.8)
    & (df['perc_of_total_cum'] <= 0.9), 
    'sales_to_customer_ABC'] = 'B'

df.loc[df['sales_to_customer_ABC'].isna()==True, 'sales_to_customer_ABC'] = 'C'

df = df.drop(columns=['perc_of_total', 'perc_of_total_cum'])

### Профит клиента

In [316]:
df = df.sort_values(by=['customers_profit'], ascending=False)

df['perc_of_total'] = df['customers_profit'] / df['customers_profit'].sum(axis=0)

df['perc_of_total_cum'] = df['perc_of_total'].cumsum(axis=0)

df.loc[df['perc_of_total_cum'] <= 0.8, 'customers_profit_ABC'] = 'A'

df.loc[
    (df['perc_of_total_cum'] > 0.8)
    & (df['perc_of_total_cum'] <= 0.9), 
    'customers_profit_ABC'] = 'B'

df.loc[df['customers_profit_ABC'].isna()==True, 'customers_profit_ABC'] = 'C'

df = df.drop(columns=['perc_of_total', 'perc_of_total_cum'])

### AOV

In [317]:
df = df.sort_values(by=['AOV'], ascending=False)

df['perc_of_total'] = df['AOV'] / df['AOV'].sum(axis=0)

df['perc_of_total_cum'] = df['perc_of_total'].cumsum(axis=0)

df.loc[df['perc_of_total_cum'] <= 0.8, 'AOV_ABC'] = 'A'

df.loc[
    (df['perc_of_total_cum'] > 0.8)
    & (df['perc_of_total_cum'] <= 0.9), 
    'AOV_ABC'] = 'B'

df.loc[df['AOV_ABC'].isna()==True, 'AOV_ABC'] = 'C'

df = df.drop(columns=['perc_of_total', 'perc_of_total_cum'])

# Forecast

In [318]:
df.head(100)

,longitude,latitude,customer,sales_to_customer,customer_sales,sales_to_customer_ABC,customers_profit,customers_profit_ABC,AOV,AOV_ABC,request_cnt_m,request_cnt_m2,request_cnt_m3,request_cnt_m_forecast
13,37.546821,55.436228,Клиент 14,1,2,C,1,C,1500.000000,A,12,8,6,NaN
51,38.475900,55.293300,Клиент 52,5,7,C,4,C,1209.497059,A,7,6,4,NaN
50,36.204764,55.709589,Клиент 51,85,119,A,17,B,1190.250000,A,3,35,35,NaN
49,37.345833,55.735083,Клиент 50,50,70,B,18,A,1171.002941,A,13,10,7,NaN
42,37.582735,55.754566,Клиент 43,90,126,A,63,A,1036.273529,A,34,45,58,NaN
41,37.756825,55.459751,Клиент 42,90,126,A,63,A,1017.026471,A,9,2,1,NaN
40,37.712867,55.648397,Клиент 41,70,98,A,25,A,997.779412,A,7,5,3,NaN
39,37.200745,56.009542,Клиент 40,70,98,A,14,B,978.532353,A,3,3,2,NaN
32,37.290172,55.440506,Клиент 33,90,126,A,32,A,843.802941,A,50,50,50,NaN
31,36.783547,55.852102,Клиент 32,90,126,A,18,B,824.555882,A,3,2,2,NaN
